머신러닝을 몰라도 "Config"에 5가지만 넣으면, 실행할 수 있게 만들었습니다. 
Even if you don't know machine learning, you can submit only 5 things in "config"

회사에서 많이 사용하게 만들었었고, 도움이 많이 될 겁니다. 
I made a lot of use of it in my company, and it will help a lot.

# Config

In [ ]:
import os

# 기본폴더(base folder)
dir_ad = '/kaggle/input/tabular-playground-series-nov-2021'
# dir_ad = os.getcwd()

# 학습데이터, predict data 파일명(train, test file name)
train = 'train.csv'
test = 'test.csv'

# 어떤것을 분석 및 데이터 추출할지 구분할 수 있는 파일명 선택 
# tag = 'train_MAU_3M_pred_MAUX_4M'← 이런 방식으로 나올 것으로 예상
tag = 'house_result'

# 모델 및 각종 산출물이 저장될 폴더
model_dir = os.path.join(dir_ad, tag + '_v')

# 레이블에 해당하는 컬럼(column corresponding to label: Y value)
target = 'target'

# 타겟팅 대상을 식별하기 위한 컬럼(Column to identify the target)
index_col = 'id'

# 평가방법 "accuracy", "roc_auc","f1" 등 선택
choice = "roc_auc"

# Model

In [ ]:
import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import ols
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

In [ ]:
df_train = pd.read_csv(os.path.join(dir_ad, train))
df_test = pd.read_csv(os.path.join(dir_ad, test))

In [ ]:
num_cols = [col for col in df_test.columns if df_test[col].dtype in ["float16","float32","float64", "int64", "int32"]]
cat_cols = [col for col in df_test.columns if df_test[col].dtype not in ["float16","float32","float64", "int64", "int32"]]

In [ ]:
def detect_outliers(data, col_name):
    outlier_indices = []
    # iterate over features(columns)
    for col in col_name:
        Q1 = np.percentile(data[col], 25)
        Q3 = np.percentile(data[col],75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR
        outlier_list_col = data[(data[col] < Q1 - outlier_step) | (data[col] > Q3 + outlier_step )].index
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
# Drop outliers
df_train = df_train.drop(detect_outliers(df_train, num_cols), axis = 0).reset_index(drop=True)

In [ ]:
def outlier_replace(data, col_name, q1=0.25, q3=0.75):
    quartile1 = data[col_name].quantile(q1)
    quartile3 = data[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    data.loc[(data[col_name] < low_limit), col_name] = low_limit
    data.loc[(data[col_name] > up_limit), col_name] = up_limit

In [ ]:
for i in num_cols:
    outlier_replace(df_train,i)
    outlier_replace(df_test, i)

In [ ]:
df_train = df_train.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
df_test = df_test.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)

In [ ]:
def count_plot(d, y, x):
    plt.figure(figsize=(12,6))
    sns.countplot(x = d[y], hue = x, data=d)
    plt.ylabel('Number of people')
    plt.title('Survival count by '+ x)

In [ ]:
over_column_name = list()

for i in num_cols:
    if (len(df_train[i].value_counts())<20):
        count_plot(df_train, target, i)
    elif (len(df_train[i].value_counts())>20):
        over_column_name.append(i)

In [ ]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [ ]:
test_col = []
delete_train = []

for i in df_test.columns:
    test_col.append(i)
for j in df_train.columns:
    if j not in test_col:
        delete_train.append(j)
delete_train.remove(target)

In [ ]:
train_col = []
delete_test = []

for i in df_train.columns:
    train_col.append(i)
for j in df_test.columns:
    if j not in train_col:
        delete_test.append(j)

In [ ]:
for delete in delete_train:
    df_train = df_train.drop([delete],axis=1)
    
for delete in delete_test:
    df_test = df_test.drop([delete],axis=1)

In [ ]:
df_train[target] = df_train[target].astype(int)
train_y = df_train[target]
train_x = df_train.drop(labels = [target],axis = 1)

In [ ]:
# Kfold로 Cross validate
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=5)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()

## Search grid for optimal parameters
rf_param_grid = {"max_depth": [10],
              "max_features": [3,20],
              "min_samples_split": [3, 10],
              "min_samples_leaf": [3, 10],
              "bootstrap": [False],
              "n_estimators" :[4000],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring = choice, verbose = 1)
gsRFC.fit(train_x, train_y)
RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [ ]:
# XGB Parameters tunning 
XGB = xgb.XGBClassifier()

## Search grid for optimal parameters
xg_param_grid = {"max_depth": [5],
              "learning_rate": [0.007],
              "eval_metric": ['auc'],
#               "n_estimators": [6000],
              "tree_method": ['gpu_hist'], 
              "gamma": [1.5],
              "subsample": [0.6],
              "max_delta_step": [3],
              "min_child_weight": [5]}

gsXGB = GridSearchCV(XGB,param_grid = xg_param_grid, cv=kfold, scoring = choice, verbose = 1)
gsXGB.fit(train_x, train_y)
XGB_best = gsXGB.best_estimator_

# Best score
gsXGB.best_score_

In [ ]:
# LR Parameters tunning 
LR = LogisticRegression()

## Search grid for optimal parameters
lr_param_grid = {"max_iter": [20000],
                 "penalty": ['l2']}


gsLR = GridSearchCV(LR,param_grid = lr_param_grid, cv=kfold, scoring = choice, verbose = 1)
gsLR.fit(train_x, train_y)
LR_best = gsLR.best_estimator_

# Best score
gsLR.best_score_

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

g = plot_learning_curve(gsRFC.best_estimator_,"RF learning curves",train_x,train_y,cv=kfold)
# g = plot_learning_curve(gsXGB.best_estimator_,"XGB learning curves",train_x,train_y,cv=kfold)
g = plot_learning_curve(gsLR.best_estimator_,"LR learning curves",train_x,train_y,cv=kfold)

In [ ]:
# 3개 모델 heat map으로 상관관계 비교
# Correlation comparison with three model heat maps

test_RFC = pd.Series(RFC_best.predict(df_test), name="RFC")
test_XGB = pd.Series(XGB_best.predict(df_test), name="XGB")
test_LR = pd.Series(LR_best.predict(df_test), name="LR")

ensemble_results = pd.concat([test_RFC,test_XGB,test_LR],axis=1)

g= sns.heatmap(ensemble_results.corr(),annot=True)

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('xgb', XGB_best),('lr', LR_best)], voting='soft', n_jobs=4)
votingC = votingC.fit(train_x, train_y)

In [ ]:
test_Result = pd.Series(votingC.predict(df_test), name=target)
df_test = pd.read_csv(os.path.join(dir_ad, test))
submission = pd.DataFrame({index_col: df_test[index_col], target: test_Result})
submission.to_csv("submission.csv",index=False)
submission.head()